# Logic gates with Neural Networks (Backpropogation)

In [1]:
import numpy as np
import pandas as pd
pd.set_option('max_colwidth', None)

In [2]:
def sigmoid(x):
    """Implementing the sigmoid function for x.
    sig(x) = 1/(1+e^-x)

    Args:
        x: input for which signmoid function needs to be calculated.

    Returns:
        the sigmoid function.
    """
    return 1 / (1 + (np.exp(-x)))

In [3]:
def sigmoid_derivative(x):
    """Implementing the sigmoid function for x.
    sig'(x) = x * (1-x)

    Args:
        x: input for which derivative of signmoid function needs to be calculated.

    Returns:
        the derivative of sigmoid function.
    """
    return sigmoid(x) * (1-sigmoid(x))

## Setting input, `X` and output, `Y_target`

In [4]:
# set the Input datasets
X = np.array([[0,0],[0,1],[1,0],[1,1]])
X

array([[0, 0],
       [0, 1],
       [1, 0],
       [1, 1]])

In [5]:
outputs = {
    'and': [[0], [0], [0], [1]], 'nand': [[1], [1], [1], [0]], 'or': [[0], [1], [1], [1]],
    'nor': [[1], [0], [0], [0]], 'xor': [[0], [1], [1], [0]], 'xnor': [[1], [0], [0], [1]]
}
input_list = [(0, 0), (0, 1), (1, 0), (1, 1)]

lg = """
█░░ █▀▀█ █▀▀▀ ░▀░ █▀▀ 　 █▀▀▀ █▀▀█ ▀▀█▀▀ █▀▀ █▀▀ 
█░░ █░░█ █░▀█ ▀█▀ █░░ 　 █░▀█ █▄▄█ ░░█░░ █▀▀ ▀▀█ 
▀▀▀ ▀▀▀▀ ▀▀▀▀ ▀▀▀ ▀▀▀ 　 ▀▀▀▀ ▀░░▀ ░░▀░░ ▀▀▀ ▀▀▀
A B | AND NAND OR NOR XOR XNOR
-------------------------------
0 0 |  0    1   0  1   0   1
0 1 |  0    1   1  0   1   0
1 0 |  0    1   1  0   1   0
1 1 |  1    0   1  0   0   1
"""
print(lg)

while True:
    try:
        output_gate = input('Enter your chosen logic gate: ').lower()
        break
    except KeyError:
        print('Invalid logic gate, try again!\n')

# set the expected output
Y_target = np.array(outputs[output_gate])
Y_target


█░░ █▀▀█ █▀▀▀ ░▀░ █▀▀ 　 █▀▀▀ █▀▀█ ▀▀█▀▀ █▀▀ █▀▀ 
█░░ █░░█ █░▀█ ▀█▀ █░░ 　 █░▀█ █▄▄█ ░░█░░ █▀▀ ▀▀█ 
▀▀▀ ▀▀▀▀ ▀▀▀▀ ▀▀▀ ▀▀▀ 　 ▀▀▀▀ ▀░░▀ ░░▀░░ ▀▀▀ ▀▀▀
A B | AND NAND OR NOR XOR XNOR
-------------------------------
0 0 |  0    1   0  1   0   1
0 1 |  0    1   1  0   1   0
1 0 |  0    1   1  0   1   0
1 1 |  1    0   1  0   0   1

Enter your chosen logic gate: xor


array([[0],
       [1],
       [1],
       [0]])

## Training

In [6]:
# assign Random weights
# 6 for hidden layer
# 3 for output layer
W1 = np.array([[0.1, 0.2, 0.3], [0.4, 0.5, 0.6]])
W2 = np.array([[0.7], [0.8], [0.9]])
print("Initial weights 1:\n", W1)
print("Initial weights 2:\n", W2)

# learning rate
lr = 0.05
print("\nLearning rate:", lr)

for epoch in range(20000):
    # forward propagation
    Z = np.dot(X, W1)
    H = sigmoid(Z)

    U = np.dot(H, W2)
    Y = sigmoid(U)

    # calculate Loss function (Mean Square error loss)
    E = Y - Y_target
    L = 1/2 * (np.power(E, 2))
#     print(f"{epoch} - Error: {E.sum()},  MSE loss: {L.sum()}")

    # backpropagation - Stage 1
    dL_dY = Y - Y_target
    dY_dU = Y * (1-Y)
    dU_dW2 = H

    dL_dW2 = np.dot(dU_dW2.T,dL_dY*dY_dU)

    # weight updates in stage 1
    W2 -= lr*dL_dW2

    # backpropagation - Stage 2
    dL_dY = Y-Y_target
    dY_dU = Y*(1-Y)
    dU_dH = W2
    dH_dZ = H*(1-H)
    dZ_dW1 = X 

    dL_dH = np.dot(dL_dY*dY_dU,dU_dH.T)
    dL_dW1 = np.dot(dZ_dW1.T,dH_dZ*dL_dH)

    # weight updates in stage 2
    W1 -= lr*dL_dW1

print("Final weights 1\n", W1)
print("Final weights 2\n", W2)

Initial weights 1:
 [[0.1 0.2 0.3]
 [0.4 0.5 0.6]]
Initial weights 2:
 [[0.7]
 [0.8]
 [0.9]]

Learning rate: 0.05
Final weights 1
 [[-1.27669418  3.25509563  5.03516485]
 [ 3.2199263  -1.3052818   5.06691904]]
Final weights 2
 [[-5.95676969]
 [-5.89124642]
 [ 8.71065791]]


## Testing

In [7]:
z_vals = []
h_vals = []
u_vals = []
y_vals = []
preds = []

for point in X:
    point = np.array(point)
    z = np.dot(point, W1)
    z_vals.append(z)

    h = sigmoid(z)
    h_vals.append(h)

    op = np.dot(h, W2)
    u_vals.append(op)

    y_cap = sigmoid(op)
    y_vals.append(y_cap)

    preds.append(y_cap[0])

print(f"LOGIC GATE: {output_gate.upper()}")
pd.DataFrame.from_dict({
    'Input1': X.reshape(-1)[::2],
    'Input2': X.reshape(-1)[1::2],
    'Z': z_vals,
    'H': h_vals,
    'U': u_vals,
    'Y': y_vals,
    'Target': Y_target.reshape(-1),
    'Prediction': preds
})

LOGIC GATE: XOR


,Input1,Input2,Z,H,U,Y,Target,Prediction
0,0,0,"[0.0, 0.0, 0.0]","[0.5, 0.5, 0.5]",[-1.5686791007710408],[0.1724047779001969],0,0.172405
1,0,1,"[3.219926297971157, -1.3052817955798506, 5.06691904357901]","[0.9615772915661411, 0.2132774414660749, 0.9937376582107277]",[1.6717443660956899],[0.8418082518876534],1,0.841808
2,1,0,"[-1.2766941797968783, 3.255095628350392, 5.035164845843059]","[0.21811347272140294, 0.962855785812643, 0.9935369174786617]",[1.682687783641075],[0.8432601100356194],1,0.843260
3,1,1,"[1.9432321181742789, 1.9498138327705414, 10.102083889422069]","[0.8747067960942035, 0.87542634075265, 0.9999590076413337]",[-1.6574783879364199],[0.16010078384135087],0,0.160101
